In [ ]:
# !sed -i 's/transformers.generation_utils/transformers.generation/' /usr/local/lib/python3.12/dist-packages/openprompt/pipeline_base.py

In [ ]:
# !pip install transformers==4.24.0

In [1]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


Removed cache: C:\Users\stdFurqan\.cache\huggingface\hub\models--bert-base-multilingual-cased
Removed cache: C:\Users\stdFurqan\.cache\huggingface\hub\models--xlm-roberta-base
Removed cache: C:\Users\stdFurqan\.cache\huggingface\hub\models--roberta-base


In [2]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import torch

print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [4]:
# ==============================
# Imports
# ==============================
import pandas as pd
import torch
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:


# ==============================
# Set random seeds for reproducibility
# ==============================
# 💡 Added this block to ensure consistent results across runs
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

    InputExample(guid=0, text_a="پروفیسر نے بات کی ہم سے۔", label=0),
    InputExample(guid=1, text_a="پروفیسر نے بات کی ہم سے۔", label=0),
    InputExample(guid=2, text_a="م خود سے چیخ رہے تھا ۔", label=0),
    InputExample(guid=3, text_a="ہم نے ہیری کو گھبرا کر پکارا تھا۔", label=0),
    InputExample(guid=4, text_a="ہیری نے خود کو کھانسی دے دیا۔", label=0),
    InputExample(guid=5, text_a="ہیری نے ہمیں کھانسی دے کر فٹ کر دیتا تھا ۔", label=0),
    InputExample(guid=6, text_a="انہوں نے اسے بنا کر غصہ دلاتا تھا ۔", label=0),
    InputExample(guid=7, text_a="انہوں نے اسے صدر بنا کر بنا دیتے تھا ۔", label=0),
    InputExample(guid=8, text_a="انہوں نے اسے تھکا دیتا تھا ۔", label=0),
    InputExample(guid=9, text_a="گاڑی سڑک پر ہارن مارتی تھی کو۔", label=0),
    InputExample(guid=10, text_a="کتا کمرے سے باہر بھونکا دیا۔", label=0),
    InputExample(guid=11, text_a="ڈائن غائب ہو کر جنگل میں گئی تھی۔", label=0),
    InputExample(guid=12, text_a="عمارت اونچی اور اونچی ہیں ۔", label=0),
    InputExample(guid=13, text_a="یہ عمارت اونچی اور اونچی ہیں ۔", label=0),
    InputExample(guid=14, text_a="اس عمارت کو اس سے زیادہ مل گیا ہے تھے ۔", label=0),
    InputExample(guid=15, text_a="یہ عمارت اس سے زیادہ ہیں ۔", label=0),

    
    InputExample(guid=16, text_a="ہمارے دوست اس تجزیے کو نہیں خریدیں گے - اگلے تجزیے کو چھوڑ دیں جو ہم تجویز کرتے ہیں ۔", label=1),
    InputExample(guid=17, text_a="ایک اور فرضی عام کاری اور میں ہار مان رہا ہوں ۔", label=1),
    InputExample(guid=18, text_a="ایک اور فرضی عام کاری یا میں ترک کر رہا ہوں ۔", label=1),
    InputExample(guid=19, text_a="ہم فعل کا جتنا زیادہ مطالعہ کرتے ہیں وہ اتنے ہی پاگل ہوتے جاتے ہیں ۔", label=1),
    InputExample(guid=20, text_a="روز روز حقائق مبہم ہوتے جا رہے ہیں ۔", label=1),
    InputExample(guid=21, text_a="میں آپ کو ایک مشروب ٹھیک کر دوں گا ۔", label=1),
    InputExample(guid=22, text_a="فریڈ نے پودوں کو چپٹا پانی دیا ۔", label=1),
    InputExample(guid=23, text_a="بل کو کھانسی آنے پر ریستوراں سے باہر نکل آیا ۔", label=1),
    InputExample(guid=24, text_a="ہم رات کو رقص کر رہے ہیں ۔", label=1),
    InputExample(guid=25, text_a="ہرمن نے دھات کے فلیٹ پر ہتھوڑا مارا ۔", label=1),
    InputExample(guid=26, text_a="ناقدین نے اسٹیج سے باہر ڈرامے کو ہنس دیا ۔", label=1),
    InputExample(guid=27, text_a="تالاب ٹھوس طور پر جم گیا ۔", label=1),
    InputExample(guid=28, text_a="بل کمرے سے باہر نکل آیا ۔", label=1),
    InputExample(guid=29, text_a="باغبان نے پھولوں کو چپٹا پانی دیا ۔", label=1),
    InputExample(guid=30, text_a="باغبان نے پھولوں کو پانی دیا ۔", label=1),
    InputExample(guid=31, text_a="بل نے باتھ ٹب کو ٹکڑوں میں توڑ دیا ۔", label=1),    
]


In [7]:
# ==============================
# Define Classes
# ==============================
classes = ["unacc", "acc"]
label_map = {"unacc": 0, "acc": 1}

# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\stdFurqan\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to r

In [8]:

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    # text = '{"placeholder":"text_a"} هذه الجملة {"mask"}.',
    text='{"placeholder":"text_a"} یہ جملہ {"mask"} ہے۔',
    tokenizer=tokenizer,
)
 


# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
        "acc": ["صحیح","درست"], 
       "unacc": ["نا درست","غلط"]
    },
    tokenizer=tokenizer,
)

In [9]:



# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# DataLoader for Training
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True  # Even though shuffled, reproducibility is preserved by seeding
)

# # ==============================
# # Fine-Tuning the Prompt Model
# # ==============================
prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(20):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")


# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\cola_dataset\final_ColA_Dev_Urdu_labeled - Sheet1.csv")
eval_dataset = [
    InputExample(guid=i, text_a=row['Urdu Sentence'], label=label_map[row['label']])
    for i, row in df.iterrows()
]


eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 Fine-tuned Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))


tokenizing: 32it [00:00, 5334.78it/s]


Epoch 1 Loss: 7.7734
Epoch 2 Loss: 5.7380
Epoch 3 Loss: 4.8178
Epoch 4 Loss: 2.8221
Epoch 5 Loss: 1.7632
Epoch 6 Loss: 1.3034
Epoch 7 Loss: 0.5537
Epoch 8 Loss: 0.2640
Epoch 9 Loss: 0.0834
Epoch 10 Loss: 0.0804
Epoch 11 Loss: 0.0256
Epoch 12 Loss: 0.0339
Epoch 13 Loss: 0.0119
Epoch 14 Loss: 0.0191
Epoch 15 Loss: 0.1016
Epoch 16 Loss: 0.0399
Epoch 17 Loss: 0.0815
Epoch 18 Loss: 0.0289
Epoch 19 Loss: 0.0275
Epoch 20 Loss: 0.0429


tokenizing: 1043it [00:00, 3139.72it/s]



📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

       unacc     0.4175    0.3851    0.4006       322
         acc     0.7346    0.7601    0.7471       721

    accuracy                         0.6443      1043
   macro avg     0.5760    0.5726    0.5739      1043
weighted avg     0.6367    0.6443    0.6401      1043

